In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['font.family'] = 'Arial'

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
import glob
import os

import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

In [ ]:
from access_biology_data import annotation, gwas_studies, meta, phenotype_collections, properties, relations
from access_literature_data import medline
from access_mixed_data import genealacart
from access_science_shared import standardizer, utils

In [ ]:
import sys
sys.path.append('./../src/')

import nar170604f_occurences as nar_attention
import nar170830f_predictions as forec
import resci_inout as rinout
import resci_tools as ret

import nar180310_mega_integrator as mega

In [ ]:
taxon_id = 9606

In [ ]:
save_images=False

In [ ]:
# def export(file_base):
#     p = '180318_detail_on_clusters/{}'.format(file_base)
    
#     ppn = p + '.png'
#     rinout.ensure_presence_of_directory(ppn)
#     ret.export_raster_image(ppn, dpi=300, insert_date_time=False)
    
#     ppd = p + '.pdf'
#     rinout.ensure_presence_of_directory(ppn)
#     ret.export_image(ppd, insert_date_time=False)

In [ ]:
ref_genes = mega.get_ref_genes()

In [ ]:
papers = mega.get_publications()

In [ ]:
tsne_frame = mega.load_layout(rotation_degrees=45)

In [ ]:
papers['enrichment_attention'] = np.log2(
    papers['attention']/ papers['attention'].mean())

In [ ]:
papers['enrichment_attention_2015'] = np.log2(
    papers['attention_2015']/ papers['attention_2015'].mean())

In [ ]:
%%time

cl = dict()
dd = dict()
ge = dict()


categs = {
    'gwas': mega.frequent_gwas,
    'gtx': mega.gtx,
#     'lof': mega.LoF,
    'duf': mega.DUF,
    'extreme_swissprot': mega.extreme_swissprot,
#     'orphan_disease': mega.orphan_disease,
    'rare_go': mega.rare_go,
    'signal_peptide': mega.signal_peptide,
    'rnai': mega.rnai_phenotypes,
    'rare_compounds': mega.rare_compounds,
    'bioplex_fame': mega.fame_in_bioplex,
    'challenged_proteins': mega.challenged_proteins,
    'detection_in_tissues': mega.detection_in_tissues,
    'detection_in_cells': mega.detection_in_cells,
    'westernblot_in_biogrid': mega.biogrid_western_blot,
    'presence_of_homologs': mega.presence_of_homologs,
    'fame_of_homologs': mega.fame_of_homologs,
#     'pi_transition': mega.pi_transition,
#     'supporting_nih_institutes': mega.supporting_nih_institutes,
    'fame_rank': mega.fame_rank
}

for k, i in categs.items():
    print(k)
    a, b, c = i()
    cl[k] = a
    dd[k] = b
    ge[k] = c

In [ ]:
# su = pd.concat(cl.values(), axis=1, join='outer').rename_axis('gene_ncbi')

In [ ]:
summary = dict()

In [ ]:
cl.keys()

In [ ]:
naming = {
    'amount_measured_amino_acids': 'amount_amino_acids',
    'minus_isoelectric_point': 'low_isolectric_point',
    'isoelectric_point': 'high_isoelectric_point',
    'SEG_fraction': 'unstructured_fraction',
    'SEG_fraction_longest': 'unstructured_covered_by_longest',
    'RADAR_fraction': 'repetitiveness', 
    'gravy_ignoring_O_and_U': 'GRAVY'
}

actor = cl['extreme_swissprot'].copy().rename(columns=naming)

actor = actor.drop('extreme_swissprot', 1)

actor = actor.loc[:, sorted(actor.columns)]

out = pd.DataFrame(index=actor.index, columns=['biophysics'])
out.loc[:, 'biophysics'] = ''

for c in actor.columns:
    f = actor.loc[:, c] == True
    out.loc[f, 'biophysics'] = out.loc[f, 'biophysics'] + c + ', '

out.loc[:, 'biophysics'] = out.loc[:, 'biophysics'].str.strip(' ,')

out = out.reindex(ref_genes).fillna('')

summary['unusual_chemistry'] = out

In [ ]:
side = dd['gtx'].copy()

In [ ]:
side.loc[:, 'gtx_level'] = side.loc[:, 'gtx_fraction'].rank(pct=True)

In [ ]:
side.loc[:, 'EBI GXA'] = ''

In [ ]:
f = side['gtx_level'] >= 0.80
side.loc[f, 'EBI GXA'] = 'within 20% most responsive in EBI-GXA'

f = side['gtx_level'] >= 0.90
side.loc[f, 'EBI GXA'] = 'within 10% most responsive in EBI-GXA'

f = side['gtx_level'] >= 0.95
side.loc[f, 'EBI GXA'] = 'within 5% most responsive in EBI-GXA'

f = side['gtx_level'] >= 0.99
side.loc[f, 'EBI GXA'] = 'within 1% most responsive in EBI-GXA'

In [ ]:
b_side = cl['rnai']
b_side = b_side[b_side['rnai_frequent']==True]

In [ ]:
c_side = dd['westernblot_in_biogrid']
c_side = c_side[c_side['biogrid_western_blot']==True]

In [ ]:
out = side[['EBI GXA']]
out = out.reindex(ref_genes)

out.loc[:, 'GenomeRNAi'] = ''
out.loc[b_side.index, 'GenomeRNAi'] = 'high occurence in GenomeRNAi'

out = out.reindex(ref_genes).fillna('')

f = out.index.isin(c_side.index)
out.loc[f, 'BioGRID'] = 'Affinity Western'

out = out.fillna('')

In [ ]:
summary['experiments'] = out

In [ ]:
side = properties.transcript_abundance_uhlen_2015_tissues().set_index('gene_ncbi')
side.columns = [x[len('uhlen_2015_cells_log10fpkm: '):] for x in side.columns]
side.columns = [x.split('_')[0] for x in side.columns]

best_tissue = side.idxmax(axis=1).dropna()

In [ ]:
side = properties.transcript_abundance_uhlen_2015_cells().set_index('gene_ncbi')
side.columns = [x[len('uhlen_2015_cells_log10fpkm: '):] for x in side.columns]
side.columns = [x.split('_')[0] for x in side.columns]

best_cells = side.idxmax(axis=1).dropna()

In [ ]:
out = pd.DataFrame(index=ref_genes, columns=['highest tissue', 'highest cells'])

In [ ]:
out['highest tissue'] = best_tissue

In [ ]:
out['highest cells'] = best_cells

In [ ]:
out = out.fillna('')

In [ ]:
summary['highest_expression'] = out

In [ ]:
he = relations.bioplex2()
papers = mega.get_publications()

he = he[he['gene_ncbi'].isin(ref_genes)]

ge = he['gene_ncbi'].unique()

h = pd.merge(
    he,
    papers[['attention']].reset_index())

hh = pd.merge(
    h,
    h,
    left_on='bioplex2_id',
    right_on='bioplex2_id',
    suffixes=('_cis', '_trans')
)

b = hh[hh['gene_ncbi_cis'] != hh['gene_ncbi_trans']
       ][['gene_ncbi_cis', 'attention_trans', 'gene_ncbi_trans']]

In [ ]:
b = b.sort_values(['gene_ncbi_cis', 'attention_trans', 'gene_ncbi_trans'], ascending=True)

In [ ]:
gi = meta.gene_info(taxon_id=9606, usecols=['gene_ncbi', 'symbol_ncbi']).rename(
    columns={'gene_ncbi': 'gene_ncbi_trans'})

In [ ]:
side_lowest = b.groupby('gene_ncbi_cis').first().reset_index()[
    ['gene_ncbi_cis', 'gene_ncbi_trans']].rename(columns={
    'gene_ncbi_cis': 'gene_ncbi', 
})

side_lowest = pd.merge(side_lowest, gi, how='left')[
    ['gene_ncbi', 'symbol_ncbi']
].set_index('gene_ncbi').rename(columns={
    'symbol_ncbi': 'least_studied_in_bioplex2'
})

In [ ]:
side_highest = b.groupby('gene_ncbi_cis').last().reset_index()[
    ['gene_ncbi_cis', 'gene_ncbi_trans']].rename(columns={
    'gene_ncbi_cis': 'gene_ncbi', 
})

side_highest = pd.merge(side_highest, gi, how='left')[
    ['gene_ncbi', 'symbol_ncbi']
].set_index('gene_ncbi').rename(columns={
    'symbol_ncbi': 'most_studied_in_bioplex2'
})

In [ ]:
out = pd.concat([side_lowest, side_highest], axis=1)

In [ ]:
out = out.reindex(ref_genes).fillna('')

In [ ]:
summary['from_bioplex'] = out

In [ ]:
# gene names for all genes
entrez_2_name = meta.gene_info(taxon_id='all', usecols=['gene_ncbi', 'symbol_ncbi'])

In [ ]:
# import homologene
hg = relations.homologene()

In [ ]:
g = hg.taxon_ncbi.unique()

In [ ]:
[print(meta.taxon_name(x), x) for x in g];

In [ ]:
hg_popular_vertebrate = hg[hg['taxon_ncbi'].isin([
    10090,
    10116,
    7955, # danio
    8364, # xenopus
    9031, # chicken
    
])]

In [ ]:
hg_popular_invertebrate = hg[hg['taxon_ncbi'].isin([
    284812,
    559292,
    6239,
    7227,
#    3702, # arabidopsis
])]

In [ ]:
hg_human = hg[hg['taxon_ncbi'].isin([
    9606
])]

In [ ]:
gene2pubmed_research = medline.gene2pubmed(
    taxon_id='all', paper_kind='research')

value_of_pubmed_id = gene2pubmed_research[
    'pubmed_id'].value_counts().to_frame().reset_index().rename(
    columns={'index': 'pubmed_id', 'pubmed_id': 'value'})

value_of_pubmed_id['value'] = 1 / value_of_pubmed_id['value']

gene2pubmed_research = pd.merge(gene2pubmed_research, value_of_pubmed_id)
extended_attention = gene2pubmed_research[[
    'gene_ncbi', 'value']].groupby('gene_ncbi').agg(sum)

In [ ]:
#

In [ ]:
side = pd.merge(
    hg_human[['homologene_group', 'gene_ncbi']],
    hg_popular_vertebrate[['homologene_group', 'taxon_ncbi', 'gene_ncbi']],
    left_on='homologene_group',
    right_on='homologene_group',
    how='left',
    suffixes=('', '_h'))

side = pd.merge(
    side,
    extended_attention.reset_index(),
    left_on='gene_ncbi_h',
    right_on='gene_ncbi',
    suffixes=('', '_model')
)

side = pd.merge(side, entrez_2_name.rename(columns={'gene_ncbi': 'gene_ncbi_h'}), how='left')

g = side[
    [
        'gene_ncbi', 
        'taxon_ncbi', 
        'value',
        'symbol_ncbi'
    ]].sort_values(['gene_ncbi', 'value', 'taxon_ncbi'])[
    ['gene_ncbi', 'taxon_ncbi', 'symbol_ncbi']
].groupby('gene_ncbi').last()

g_vertebrate = g.reset_index().rename(columns={
    'taxon_ncbi': 'popular vertebrate',
    'symbol_ncbi': 'popular vertebrate gene'
})

In [ ]:
side = pd.merge(
    hg_human[['homologene_group', 'gene_ncbi']],
    hg_popular_invertebrate[['homologene_group', 'taxon_ncbi', 'gene_ncbi']],
    left_on='homologene_group',
    right_on='homologene_group',
    how='left',
    suffixes=('', '_h'))

side = pd.merge(
    side,
    extended_attention.reset_index(),
    left_on='gene_ncbi_h',
    right_on='gene_ncbi',
    suffixes=('', '_model')
)

side = pd.merge(side, entrez_2_name.rename(columns={'gene_ncbi': 'gene_ncbi_h'}), how='left')

g = side[
    [
        'gene_ncbi', 
        'taxon_ncbi', 
        'value',
        'symbol_ncbi'
    ]].sort_values(['gene_ncbi', 'value', 'taxon_ncbi'])[
    ['gene_ncbi', 'taxon_ncbi', 'symbol_ncbi']
].groupby('gene_ncbi').last()

g_invertebrate = g.reset_index().rename(columns={
    'taxon_ncbi': 'popular invertebrate',
    'symbol_ncbi': 'popular invertebrate gene'
})

In [ ]:
u = hg['taxon_ncbi'].unique()

In [ ]:
namer = dict()
for uu in u:
    namer[uu] = meta.taxon_name(uu)

In [ ]:
g_invertebrate['popular invertebrate'] = g_invertebrate['popular invertebrate'].replace(
    namer)

In [ ]:
g_vertebrate['popular vertebrate'] = g_vertebrate['popular vertebrate'].replace(
    namer)

In [ ]:
g_invertebrate['popular invertebrate'].value_counts()

In [ ]:
out = pd.concat([
    g_invertebrate.set_index('gene_ncbi'),
    g_vertebrate.set_index('gene_ncbi'),    
], axis=1).reindex(ref_genes).fillna('')

In [ ]:
taxon_abbreviations = {
    'Drosophila melanogaster': 'D. melanogaster',
    'Saccharomyces cerevisiae S288c': 'S. cerevisiae',
    'Caenorhabditis elegans': 'C. elegans',
    'Schizosaccharomyces pombe 972h-': 'S. pombe',
    'Mus musculus': 'M. musculus',
    'Rattus norvegicus': 'R. norvegicus',
    'Danio rerio': 'D. rerio',
    'Gallus gallus': 'G. gallus',
    'Xenopus tropicalis': 'X. tropicalis'
}

In [ ]:
out = out.replace(taxon_abbreviations)

In [ ]:
summary['model_organisms'] = out

In [ ]:
out['popular invertebrate'] = out['popular invertebrate'] + ' (' + out['popular invertebrate gene'] + ')'
out['popular vertebrate'] = out['popular vertebrate'] + ' (' + out['popular vertebrate gene'] + ')'
out = out.replace(' ()','')

In [ ]:
out = out[['popular invertebrate', 'popular vertebrate']]

In [ ]:
actor = dd['duf']

out = pd.DataFrame(index=actor.index, columns=['domain(s) of unknown function'])
out.loc[:, 'domain(s) of unknown function'] = ''

for c in actor.columns:
    f = actor.loc[:, c] == True
    out.loc[f, 'domain(s) of unknown function'] = out.loc[f, 'domain(s) of unknown function'] + c + ', '

out.loc[:, 'domain(s) of unknown function'] = out.loc[:, 'domain(s) of unknown function'].str.strip(' ,')

out = out.reindex(ref_genes).fillna('')

summary['duf'] = out

In [ ]:
def get_ref_genes():
    return ref_genes

In [ ]:
ebi_gwas = gwas_studies.ebi_gwas()

f = ebi_gwas['MAPPED_GENE'].str.contains('[;,-]') == True
gwas = ebi_gwas.loc[
    ~f,
    ['MAPPED_GENE', 'DISEASE/TRAIT', 'PVALUE_MLOG', 'pubmed_id']].rename(
    columns={
        'MAPPED_GENE': 'symbol_ambiguous',
        'DISEASE/TRAIT': 'trait',
        'PVALUE_MLOG': 'log_pvalue'
    }
)

gwas = pd.merge(
    gwas,
    meta.gene_info(taxon_id=9606, usecols=[
                   'symbol_ncbi', 'gene_ncbi']),
    left_on='symbol_ambiguous',
    right_on='symbol_ncbi',
    how='inner'
).drop('symbol_ambiguous', axis=1).drop('symbol_ncbi', axis=1)

gwas = gwas[gwas['gene_ncbi'].isin(get_ref_genes())]



In [ ]:

ge = sorted(gwas['gene_ncbi'].unique())

gwas = gwas.sort_values('log_pvalue', ascending=False)
gwas = gwas.drop_duplicates(
    ['trait', 'pubmed_id', 'gene_ncbi'],
    keep='first')

studies_per_phenotype = gwas[
    ['pubmed_id', 'trait']].drop_duplicates()[
    'trait'].value_counts()

required_studies = 10
important_gwas = gwas.loc[
    (
        gwas['trait'].isin(
            studies_per_phenotype[
                studies_per_phenotype >= required_studies].index)), :

][['pubmed_id', 'trait', 'gene_ncbi']].drop_duplicates()

In [ ]:
he = pd.merge(
    important_gwas.groupby(
        ['trait', 'gene_ncbi']).size(
    ).reset_index().rename(columns={0: 'records'}),
    studies_per_phenotype.to_frame(
        'studies').reset_index().rename(columns={'index': 'trait'}))

he.loc[:, 'fraction_of_gwas_studies'] = he['records'] / he['studies']

In [ ]:
actor = he[he['fraction_of_gwas_studies']>0.2][['trait', 'gene_ncbi']]

In [ ]:
actor.loc[:, 'present'] = True

In [ ]:
actor = actor.pivot(index='gene_ncbi', columns='trait', values='present')

In [ ]:
actor = actor.fillna(False)

In [ ]:
actor = actor.loc[:, sorted(actor.columns)]

out = pd.DataFrame(index=actor.index, columns=['strong gwas'])
out.loc[:, 'strong gwas'] = ''

for c in actor.columns:
    f = actor.loc[:, c] == True
    out.loc[f, 'strong gwas'] = out.loc[f, 'strong gwas'] + c + ', '

out.loc[:, 'strong gwas'] = out.loc[:, 'strong gwas'].str.strip(' ,')

out = out.reindex(ref_genes).fillna('')

summary['strong_gwas'] = out

In [ ]:
gene_info = meta.gene_info(9606, usecols=['gene_ncbi', 'symbol_ncbi', 'dbXrefs'])
f = gene_info['dbXrefs'].str.contains('Ensembl:')
gene_info.loc[f, 'gene_ensembl'] = gene_info.loc[f, 'dbXrefs'].str.extract('Ensembl:(ENSG[0-9]*)', expand=False)
gene_info = gene_info[['gene_ncbi', 'symbol_ncbi', 'gene_ensembl']]

In [ ]:
gene_info = gene_info[gene_info['gene_ncbi'].isin(ref_genes)].set_index('gene_ncbi')

In [ ]:
summary.keys()

In [ ]:
master = pd.concat(
[
    gene_info,
    summary['highest_expression'],
    summary['experiments'],
    summary['model_organisms'],
    summary['strong_gwas'],
    summary['from_bioplex'],
    summary['unusual_chemistry'],
    summary['duf']
], axis=1
)

In [ ]:
master.columns

In [ ]:
master = master.rename(columns={
    'popular invertebrate': 'most studied popular invertebrate',
    'popular vertebrate': 'most studied popular vertebrate',
    'least_studied_in_bioplex2': 'least studied gene in same bioplex2',
    'most_studied_in_bioplex2': 'most studied gene in same bioplex2',
    'biophysics': 'top 1% of biophysics'
})

In [ ]:
master = master.fillna('')

In [ ]:
actor = properties.compartment_itzhak_2016_determined().set_index('gene_ncbi')
actor = actor.loc[:, sorted(actor.columns)]

out = pd.DataFrame(index=actor.index, columns=['localization'])
out.loc[:, 'localization'] = ''

for c in actor.columns:
    f = actor.loc[:, c] == True
    out.loc[f, 'localization'] = out.loc[f, 'localization'] + c[len('Itzhak2016_Compartment Prediction: '):] + ', '

out.loc[:, 'localization'] = out.loc[:, 'localization'].str.strip(' ,')
out = out.reindex(ref_genes).fillna('')
# summary['strong_gwas'] = out
out = out.reset_index()

master = pd.merge(master.reset_index(), out, how='left').fillna('')

In [ ]:
swiss_signalp = properties.signalp_swissprot(taxon_id)
trembl_signalp = properties.signalp_trembl(taxon_id)

genes_with_signal_peptide = set(swiss_signalp[swiss_signalp['SignalP_swissprot: cleaved']==1]['gene_ncbi']).union(
set(trembl_signalp[trembl_signalp['SignalP_trembl: cleaved']==1]['gene_ncbi']))

f = master['gene_ncbi'].isin(genes_with_signal_peptide)
master.loc[f, 'localization'] = master.loc[f, 'localization'] + ', (secreted, signal peptide)'
master['localization'] =master['localization'].str.strip(", ")

In [ ]:
def export_table(file_base, df):
    p = rinout.get_internal_path(
        '180327_gene_recommendations/{}.xlsx'.format(file_base)
    )
    rinout.ensure_presence_of_directory(p)
    ret.export_full_frame(p, df, insert_date_time=False, save_index=True)

In [ ]:
export_table('entry_points', master.set_index('gene_ncbi'))

In [ ]:
# plot strongest for ulcerine colitis

he = he.sort_values('fraction_of_gwas_studies', ascending=False)

agg = []
for t in he['trait'].unique():
    hef = he[he['trait']==t][['gene_ncbi', 'fraction_of_gwas_studies']]
    hef = hef.set_index('gene_ncbi')
    hef['pct'] = hef['fraction_of_gwas_studies'].rank(pct=True)
    hef = hef[['pct']]
    hef.loc[:, 'trait'] = t
    hef = hef.reset_index()
    agg.append(hef)

h = pd.concat(agg)

he[he['gene_ncbi']==55765]

he[he['trait']=='Ulcerative colitis']

